# `FC.py` |  cofound regressions

In [22]:
# Load libraries
import sys
import os
import glob
import numpy as np
import nibabel as nib
from sklearn.linear_model import LinearRegression
import warnings

# Load data from single subject
funcDir='/data_/mica3/BIDS_MICs/derivatives/micapipe/sub-PX062/ses-01/func/desc-se_task-rest_acq-AP_bold'
func_lab='_space-func_desc-se'
# Load confound files
os.chdir(funcDir+'/volumetric/')
x_spike = " ".join(glob.glob(funcDir+'/volumetric/'+'*spikeRegressors_FD.1D'))
x_dof = " ".join(glob.glob(funcDir+'/volumetric/*'+func_lab+'.1D'))
# x_refmse = " ".join(glob.glob(funcDir+'/volumetric/'+'*metric_REFMSE.1D'))
x_fd = " ".join(glob.glob(funcDir+'/volumetric/'+'*metric_FD*'))
x_csf = " ".join(glob.glob(funcDir+'/volumetric/'+'*CSF*'))
x_wm = " ".join(glob.glob(funcDir+'/volumetric/'+'*WM*'))
x_gs = " ".join(glob.glob(funcDir+'/volumetric/'+'*global*'))

# define functions
def expand_dim(Data):
    if Data.ndim == 1:
        Data = np.expand_dims(Data, axis=1)
    return Data

# Set paths to files
dof = np.loadtxt(x_dof)
csf = expand_dim(np.loadtxt(x_csf))
wm = expand_dim(np.loadtxt(x_wm))
gs = expand_dim(np.loadtxt(x_gs))
spike = expand_dim(np.loadtxt(x_spike))

ones = np.ones((wm.shape[0], 1))
mdl = np.append(np.append(np.append(np.append(ones, dof, axis=1), wm, axis=1), csf, axis=1), gs, axis = 1)

# load cortica data
os.chdir(funcDir+'/surfaces/')
x_lh = " ".join(glob.glob(funcDir+'/surfaces/'+'*space-conte69-32k_lh_10mm*'))
x_rh = " ".join(glob.glob(funcDir+'/surfaces/'+'*space-conte69-32k_rh_10mm*'))
lh_data = nib.load(x_lh)
lh_data = np.squeeze(lh_data.get_fdata())
rh_data = nib.load(x_rh)
rh_data = np.squeeze(rh_data.get_fdata())

# Reformat data
data = []
data = np.transpose(np.append(lh_data, rh_data, axis=0))
n_vertex_ctx_c69 = data.shape[1]
del lh_data
del rh_data

## Previous implementation: OLS GLM
## $$ X_{regresors\{696,10\}} \sim Y_{timeSeries\{695,64k\}} \cdot \beta_{coef\{10,64k\}} + I_{\{10\}}$$

In [23]:
# X ~ Y 
slmXY = LinearRegression().fit(data, mdl)

## Residuals NO intercept
## $$ Residuals = Y_{timeSeries\{695,64k\}} - ( X_{regresors\{696,10\}} \cdot \beta_{coef\{10,64k\}}  ) $$

In [24]:
# residuals NO intercep
res_XY = data - np.dot(mdl, slmXY.coef_)

## Residuals with intercept
## $$Residuals = Y_{timeSeries\{695,64k\}} - ( X_{regresors\{696,10\}} \cdot \beta_{coef\{10,64k\}} + I_{\{10\}} ) $$
is not possible bc the intercep is 10.

In [25]:
# Residuals with intercept
residual_XY = data - (np.dot(mdl, slmXY.coef_) + slmXY.intercept_)

ValueError: operands could not be broadcast together with shapes (695,64984) (10,) 

## Full updated model  
## $$ Y_{timeSeries\{695,64k\}} \sim X_{regresors\{696,10\}} \cdot \beta_{coef\{10,64k\}}^T + I_{64k}$$

In [26]:
# Y ~ X (new stuff)
slmYX = LinearRegression().fit(mdl, data)

## Residuals NO intercept
## $$ Residuals = Y_{timeSeries\{695,64k\}} - (X_{regresors\{696,10\}} \cdot \beta_{coef\{10,64k\}}^T)$$

In [27]:
# residuals NO intercept
res_YX = data - np.dot(mdl, slmYX.coef_.T)

## Residuals with intercept
## $$ Residuals = Y_{timeSeries\{695,64k\}} - (X_{regresors\{696,10\}} \cdot \beta_{coef\{10,64k\}}^T + I_{1,64k}^T)$$

In [28]:
# residuals with intercept
res_YXI =  data - (np.dot(mdl, slmYX.coef_.T) + slmYX.intercept_.T)

## Residuals with function `predict`

In [29]:
# residuals with function predict(X)
residual = (data - slmYX.predict(mdl))

## Residuals function is EQUAL to residuals with intercept
## AND same results as using `R`: `residuals(lm(y~x))`

In [30]:
# residuals function is EQUAL to residuals with intercept
np.array_equal(res_YXI, residual)

True

# Similarity with and without intercept 

In [31]:
# residuals function is NOT equal to dot product of coefficients*X
np.array_equal(res_YX, residual)

# Are the methods the same?? almost
res_corr = np.zeros(695)
for i in range(0,695):
    res_corr[i] = np.corrcoef(res_YX[i,:], residual[i,:])[0,1]
np.mean(res_corr)

0.9999999349304176

# Similarity between `X~Y` and `Y~X` residuals

In [32]:
# Are the methods the same?? almost
res_diff = np.zeros(695)
for i in range(0,695):
    res_diff[i] = np.corrcoef(residual[i,:], res_XY[i,:])[0,1]
np.mean(res_diff)

0.9493226510068407